## Executor
It executes a command (`Runnable`) in the future - can be using a new `Thread`, a pooled thread or calling thread. It provides a standard means of decoupling *task submission* from *task execution*. The interface is defined as:

In [ ]:
public interface Executor {
    void execute(Runnable command);
}

A very simplistic implementaion of an executor can be:

In [ ]:
// Executes the runnable in the calling thread
class SimpleExecutor implements Executor {

    @Override
    public void execute(Runnable command) {
        command.run();
    }
}

Another executor which runs tasks in new thread can be:

In [ ]:
class ThreadPerTaskExecutor implements Executor {

    @Override
    public void execute(Runnable command) {
        new Thread(command).run();
    }
}

Decoupling submission from execution lets us easily specify, and subsequently change without great difficulty, the *execution policy* for tasks. It defines things such as:
- what thread will be used to execute tasks
- what order should tasks be executed (LIFO, FIFO, priority based)
- how many tasks will be executing concurrently
- how many tasks will wait for execution
- if a task has to be rejected, which one should it be?
- action taken before and after executing a task

## ExecutorService
Builds upon `Executor` and provides many additional functionality to manage termination and methods that can produce a `Future` for tracking progress of one or more asynchronous tasks. For lifecycle management, the following methods are available:

In [ ]:
public interface ExecutorService implements Executor {
    void shutdown();
    List<Runnable> shutdownNow();
    boolean isShutdown();
    boolean isTerminated();
    boolean awaitTermination(long timeout, TimeUnit unit) throws InterruptedException;
    // other methods
}

A simple usage:

In [ ]:
ExecutorService executor = Executors.newFixedThreadPool(10);
Runnable runnable = () -> {
    for( int i = 0; i < 10; i++) {
        System.out.println(Thread.currentThread().getName() + " " + i);
        try {
            Thread.sleep(1000);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    }
};

executor.execute(runnable);

### ExectorService Termination
The above `ExecutorService` utilises a thread pool with constant number of active threads. Thus the program doesn't end even though the `Runnable` passed to it gets completed. There are other variants of `ExecutorService` which dispose off threads once done (after waiting for predefined time duraiton), which means the program terminates once the tasks passed to the `ExecutorService` complete (as there are no active threads left).

In [ ]:
ExecutorService executor = new ThreadPoolExecutor(0, 3, 1, TimeUnit.SECONDS, new LinkedBlockingQueue<>());

Therefore, the correct way is to signal to the `ExecutorService` that we are done passing new tasks to it, and it should terminate once done. The following methods are available:  
- `shutdown` method is meant for graceful shutdown: no new tasks are accepted but previously submitted tasks are allowed to complete — including those that have not yet begun execution.  
- `shutdownNow` method initiates abrupt shutdown: it *attempts* to cancel running tasks, and does not start queued tasks.

Tasks submitted to an `ExecutorService` after it has been shutdown are handled by the *rejected execution handler*. `ExecutorService` thereby offers a way to cancel tasks: tasks that have been submitted but not yet started can always be cancelled, and tasks that have started can sometimes be cancelled if they are responsive to interruption.

In order to understand task cancellation better, we need to understand that Java doesn't provide a mechanism for safely forcing a thread to stop what it is doing. Why? Because abruptly stopping a thread is inherently unsafe. Stopping a thread makes it release all its monitors, so any object protected by this monitor may be in an inconsistent state and other objects waiting for lock would now access this object in an inconsistent state. Consider the following example:

In [ ]:
class Shared {
    private int a, b;
    public Shared(int a, int b){
        this.a = a; this.b = b;
    }
    public synchronized void swap(){
        int temp = a;
        a = b; // What if the Thread A having lock on Shared, 
               // is stopped here? We end up in an inconsistent state
        b = temp;
    }
}

As discussed in thread basics, we can arrange for a thread to stop by:
- introducing a cancellation flag
- interrupting a running thread

In both cases, the task passed to `ExecutorService` should be designed to respond to cancellation or interruption. `shutdownNow` interrupts all the running threads by calling `thread.interrupt()`. Many methods in Java respond to interruption, for example:
- `Object.wait`
- `Thread.sleep`, `Thread.join`
- Many methods in `java.util.concurrent`

There are other scenarios that don't respond to interruption, like:
- Synchronous socket I/O: server apps reading and writing to sockets. The `read` and `write` methods are not interruptible, but closing the underlying socket unblocks the blocking `read` and `write`, by throwing `SocketException`
- Synchronous I/O in `java.nio`
- Lock acquisition: if a thread is waiting to acquire an intrinsic lock, we can't do anything other than waiting. Use the `Lock` class' `lockInterruptibly` method.

In [ ]:
class SocketReader extends Thread {
    private Socket socket;
    private InputStream input;
    
    public SocketReader (Socket socket) {
       this.socket = socket;
       input = socket.getInputStream();
    }
    
    public void run() {
        try{
            byte[] buffer = new byte[BUF_SIZE];
            while(true){
                int c = input.read(buffer);
                if(c < 0){
                    break;
                } else if (c > 0){
                    processBuffer(buffer, c);
                }
            }
        } catch (IOException e) { 
            //... 
        }
    }
    
    public void interrupt(){
        try {
            socket.close();  // This would bring the thread out from being blocked by read
        } catch (IOException e) { 
            //... 
        } finally {
            super.interrupt();
        }
    }
}

Further, interrupting a thread which doesn't respond to interruptions has no visible effect:

In [ ]:
public class InfiniteThread extends Thread {
    public void run() {
        while(true)
            System.out.println("Running...");
    }
}

// calling infiniteThrad.interrupt() will have no
// effect since run block has no interruption points

Recommended way to terminate `ExecutorService`:

In [ ]:
service.shutdown(); // Signal we are done submitting tasks and it is ok to shutdown once all tasks are done
try {
    // Wait for 10 seconds for all tasks to complete
    if (!service.awaitTermination(10000, TimeUnit.MILLISECONDS)) {
        // If tasks are not done in 60 seconds, try to forcefully terminate all tasks
        service.shutdownNow();
    }
} catch (InterruptedException e) {
    service.shutdownNow();
}

Starting Java 21, `ExecutorService` implements `AutoCloseable`, thus we can do:

In [ ]:
try (ThreadPoolExecutor executor =  (ThreadPoolExecutor) Executors.newFixedThreadPool(4)) {
    executor.submit(() -> { ... });
}

### Callable and Future
`Callable` and `Future` provide better mechanics to control lifecycle of a task. `Callable` is a task that returns a result and may throw an exception. It is defined as:

In [1]:
@FunctionalInterface
public interface Callable<V> {
    V call() throws Exception;
}

To create a callable which returns nothing, use `Callable<Void>` variant.  

`Future` on the other hand, represents the result of an asynchronous computation. It provides methods to test whether the task has completed or been cancelled, retrieve its result, and cancel the task. It is defined as:

In [ ]:
public interface Future<V> {
    boolean cancel(boolean mayInterruptIfRunning);
    boolean isCancelled();
    boolean isDone();
    V get() throws InterruptedException, ExecutionException;
    V get(long timeout, TimeUnit unit) throws InterruptedException, ExecutionException, TimeoutException;
}

A `Future` can be in one of the following states:

<img src="images/future_lifecycle.png" width=700 height=auto />

If an exception occurs while processing the task, it is wrapped in `ExecutionException`, the underlying exception can be retreived using `getCause` method. If we cancel a future, we get `CancellationException`, if we are waiting for it to complete:

In [ ]:
ExecutorService executor = Executors.newFixedThreadPool(5);
Future<String> f = executor.submit(() -> {
   throw new RuntimeException("Hola!");
});

executor.shutdown();
try {
    f.get();
} catch (ExecutionException e) {
    System.out.println(e.getCause()); // Hola!
}

The `cancel` method fails if the task has already completed, has already been cancelled, or could not be cancelled for some other reason. If successful, and this task has not started when cancel is called, this task should never run. If the task has already started, then the `mayInterruptIfRunning` parameter determines whether the thread executing this task should be interrupted in an attempt to stop the task. 

`ExecutorService` provides multiple ways to create a `Future` to describe a task:

In [ ]:
public interface ExecutorService implements Executor {
    <T> Future<T> submit(Callable<T> task);
    <T> Future<T> submit(Runnable task, T result); // Future returns result once task completes successfully
    Future<?> submit(Runnable task); // Future returns null once task completes successfully
    <T> List<Future<T>> invokeAll(Collection<? extends Callable<T>> tasks) throws InterruptedException;
    <T> List<Future<T>> invokeAll(Collection<? extends Callable<T>> tasks, long timeout, TimeUnit unit) throws InterruptedException;
    <T> T invokeAny(Collection<? extends Callable<T>> tasks) throws InterruptedException, ExecutionException;
    <T> T invokeAny(Collection<? extends Callable<T>> tasks, long timeout, TimeUnit unit) 
        throws InterruptedException, ExecutionException, TimeoutException;
    // other methods
}

The `get` method is a blocking call, so how do we act on a future (from a list of multiple futures) as soon as it is available? A crude way would be to get result with timeout 0 and continuously keep polling. This is a crude implementation:

In [ ]:
ExecutorService executorService = Executors.newFixedThreadPool(5);
List<Future<Long>> futureList = new ArrayList<>(5);
for (int i = 0; i < 5; i++) {
    futureList.add(executorService.submit(getRandomSleepCallable()));
}
executorService.shutdown();

while (true) {
    Iterator<Future<Long>> iterator = futureList.iterator();
    while (iterator.hasNext()) {
        Future<Long> future = iterator.next();
        if (future.isDone()) {
            iterator.remove();
        }

        try {
            System.out.println("Completed thread that slept for: " + future.get(0, TimeUnit.SECONDS));
        } catch (TimeoutException e) {
            // Do nothing
        }

    }

    if (futureList.isEmpty()) {
        break;
    }
}

The better way is to use `CompletionService` defined as:

In [ ]:
// Combines functionality of a Executor and a BlockingQueue
public interface CompletionService<V> {
    Future<V> submit(Callable<V> task);
    Future<V> submit(Runnable task, V result); // result is what is returned upon successful completion
    Future<V> take() throws InterruptedException; // Retrieves and removes the Future representing the next
                                                  // completed task, waiting if none are yet present.
    Future<V> poll(); // Retrieves and removes the Future representing the next completed task, or null if none are present.
    Future<V> poll(long timeout, TimeUnit unit) throws InterruptedException; // Waits in this case
}

In [ ]:
ExecutorService executorService = Executors.newFixedThreadPool(5);
CompletionService<Long> completionService = new ExecutorCompletionService<>(executorService);
for (int i = 0; i < 5; i++) {
    completionService.submit(getRandomSleepCallable());
}
for (int i = 0; i < 5; i++) {
    System.out.println("Completed thread that slept for: " + completionService.take().get());
}

## Thread Pools
Independent and homogenous tasks are most suited for being distribted to multiple threads in a thread pool. However, the following tasks need special consideration:
- Dependent tasks: those that depend upon other tasks in the thread pool - we need to implicitly create constraints on the execution policy
- Tasks that use `ThreadLocal`: executors are free to use whichever thread from the pool to execute tasks
- Long running tasks.

We can see below how dependent tasks can lead to deadlock. As an example consider two tasks A and B, where B is submitted by  A and A waits for B to get completed. In a single thread executor this will always deadlock.

In [ ]:
// Important to notice that this is a single thread executor
ExecutorService executor = Executors.newSingleThreadExecutor();

public class RenderPageTask implements Callable<String> {
    public String call() throws Exception {
        Future<String> header, footer;
        
        header = executor.submit(new LoadFileTask("header.html"));
        footer = executor.submit(new LoadFileTask("footer.html"));
        
        String body = renderBody();
        
        return header.get() + body + footer.get();
    }
}

Tasks may also depend upon external resources, for example, JDBC Connections. If for example we have 15 tasks being executed and we have only 10 connections in JDBC connection pool, 5 tasks will just be waiting. In this case, we have unnecessarily large number of threads in the Thread pool.

We can also face the reverse situation if we have long running task. Long running task will eventually saturate the pool preventing shorter task from getting executed. This may be a sign to increase thread pool size.

**Thread Pool Size:** depends upon the type of task that will be submitted. There is no exact science dictating thread pool size, we just need to avoid the extremes. Keep the thread count high and all the threads compete for the limited resource. Keep the thread count low and it would lead to low resource utilization due to cores going unused. Typically:
- for compute intensive tasks, $N_{threads} = N_{cpu} + 1$
- for I/O intensive task, we first need to compute wait time vs compute time for task. In this case we use $N_{threads} = N_{cpu}\times U(1 + \frac{W}{C})$ where $U$ is target CPU utilization and $\frac{W}{C}$ is ratio of wait time to compute time

### ThreadPoolExecutor
An `ExecutorService` that executes each submitted task using one of possibly several pooled threads. It provides multiple constructors the most important (illustrating its configrations) is:

In [ ]:
public ThreadPoolExecutor(int corePoolSize,    // maintain these many threads (even if there are no tasks)
                          int maximumPoolSize, // maximum number of threads can be active as once
                          long keepAliveTime,  // thread alive for more than this time become eligible to be terminated
                          TimeUnit unit,     
                          BlockingQueue<Runnable> workQueue, // Holds tasks awaiting execution
                          ThreadFactory threadFactory,
                          RejectedExecutionHandler handler) { /* ... */}

**Work Queue:** `BlockingQueue` argument holds tasks that are waiting to be executed. There are three approaches:
- unbounded queue: keep adding newly arriving tasks to the queue. Queue will expand as the tasks increase.
- bounded queue: fixed room. What happens when the queue is full and new tasks arrive? Depends upon the saturation policy. In this case the queue  size and threadpool size must be in sync. 
- synchronous handoff: bypass queueing entirely and instead handoff that task to the worker thread. How does this work if all the worker threads are busy? If core pool size is less than max pool size, new thread will be created, else it depends upon the saturation policy.

Java offers multiple implementations:
- `LinkedBlockingQueue` : unbounded queue
- `ArrayBlockingQueue` : bounded queue, need to specify its size
- `PriorityBlockingQueue` : unbounded, same priority as `PriorityQueue`
- `SynchronousQueue` : not a pure queue, contains a single item

The below example illustrates the working of blocking queues:

In [ ]:
public class BlockingQueueDemo {
    public static void main(String[] args) {
        BlockingQueue<Integer> queue = new LinkedBlockingQueue<>();

        Producer p = new Producer(queue);
        Consumer c = new Consumer(queue);

        new Thread(p).start();
        new Thread(c).start();
    }
}

class Producer implements Runnable {
    private BlockingQueue<Integer> queue;

    public Producer(BlockingQueue<Integer> queue) {
        this.queue = queue;
    }

    @Override
    public void run() {
        // Put 100 even numbers in queue
        for (int i = 0; i < 100; i++) {
            try {
                queue.put(i * 2);
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
    }

}

class Consumer implements Runnable {
    private BlockingQueue<Integer> queue;

    public Consumer(BlockingQueue<Integer> queue) {
        this.queue = queue;
    }

    @Override
    public void run() {
        // Consume 100 integers
        for (int i = 0; i < 100; i++) {
            try {
                System.out.println(queue.take());
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
    }

}

**Saturation Policy:** when the work queue fills up, saturation polcy kicks in (also kicks in when task is submitted to an `Executor` that has been shut down). The saturation policy of a `ThreadPoolExecutor` is set by calling `setRejectedExecutionHandler`. Several implementations of `RejectedExecutionHandler` are available:
- `AbortPolicy`: causes execute to throw unchecked `RejectedExecutionException`
- `DiscardPolicy`: silently discards the submitted task
- `DiscardOldestPolicy`: discards the task that was going to be executed next and queue the newly added task
- `CallerRunsPolicy`: neither discard, nor throw the exception. Execute the submitted task not in the threadpool, but in the thread that submitted the task

**Thread Factories:** is defined as:

In [ ]:
public interface ThreadFactory {
    Thread newThread(Runnable r);
}

the default thread factory used by `ThreadPoolExecutor` creates a new, nondaemon thread with no special configuration. Specifying a `ThreadFactory` helps us specify how exactly a thread (to be added in threadpool) will be created. Why do want to do that?
- increase thread priority
- add custom logging
- give threads custom name, etc

In [ ]:
public class CustomThreadFactory implements ThreadFactory {
    private final String poolName;
    
    public CustomThreadFactory(String poolName) {
        this.poolName = poolName;
    }
    
    public Thread newThread(Runnable r) {
        return new CustomThread(r, poolName);
    }
}

class CustomThread extends Thread {
    public static final String DEFAULT_NAME = "CustomThread";
    private static volatile boolean debugLifecycle = false;
    private static final AtomicInteger created = new AtomicInteger();
    private static final AtomicInteger alive = new AtomicInteger();
    private static final Logger logger = Logger.getAnonymousLogger();
    
    public CustomThread(Runnable r){
        this(r, DEFAULT_NAME);
    }
    
    public CustomThread(Runnable r, String poolName){
        super(r, name + "_" + created.incrementAndGet());
        setUncaughtExceptionHandler(
            new Thread.UncaughtExceptionHandler() {
                public void uncaughtException(Thread t, Throwable e) {
                    loggger.log(Level.SEVERE, "Uncaught in thread " + t.getNme(), e);
                }
            }
        );
    }
        
    public void run() {
        boolean debug = debugLifecycle;
        if(debug) logger.log(Level.FINE, "Created " + getName());
        try {
            alive.incrementAndGet();
            super.run();
        } finally {
            alive.decrementAndGet();
            if(debug) logger.log(Level.FINE, "Exiting " + getName());
        }
    }
    
    public static int getThreadsCreated() { return created.get(); }
    public static int getThreadsAlive() { return alive.get(); }
    public static int getDebug() { return debugLifecycle; }
    public static int setDebug(boolean b) { debugLifecycle = b }
}

The `Executors` class provides multiple different variants of `ExecutorService` utilizing threadpools:

In [ ]:
// --- Fixed Thread Pool ---
ThreadPoolExecutor fTP = (ThreadPoolExecutor) Executors.newFixedThreadPool(10);
/* Equivalent to:
new ThreadPoolExecutor(nThreads, nThreads,
                       0L, TimeUnit.MILLISECONDS,
                       new LinkedBlockingQueue<Runnable>()); */

// --- Cached Thread Pool ---
ThreadPoolExecutor cTP = (ThreadPoolExecutor) Executors.newCachedThreadPool();
/* Equivalent to:
new ThreadPoolExecutor(0, Integer.MAX_VALUE,
                       60L, TimeUnit.SECONDS,
                       new SynchronousQueue<Runnable>()); */

// --- Single Thread Executor ---
ThreadPoolExecutor sTP = (ThreadPoolExecutor) Executors.newSingleThreadExecutor();
/* Equivalent to:
new ThreadPoolExecutor(1, 1,
                       0L, TimeUnit.MILLISECONDS,
                       new LinkedBlockingQueue<Runnable>())) */

**Extension Hooks:** three methods available that we can override:
- `beforeExecute(Thread t, Runnable r)`: invoked prior to executing the given `Runnable` in the given thread. This method is invoked by thread `t` that will execute task `r`
- `afterExecute(Runnable r, Throwable t)`: invoked upon completion of execution of the given `Runnable`. If the task ended with an exception, `t` would be non-null
- `terminated()`: invoked when the Executor has terminated.

Example usage which logs how long each task took:

In [ ]:
class TimeLogExecutor extends ThreadPoolExecutor {
    private static final Logger LOGGER = LoggerFactory.getLogger(TimeLogExecutor.class);
    
    private ThreadLocal<Long> startNanos = new ThreadLocal();
    
    protected void beforeExecute(Thread t, Runnable r) {
        super.beforeExecute(t, r);
        // Record the start time
        long nanos = System.nanoTime();
        startNanos.set(nanos);

        LOGGER.info("Thread {} started task {} at {}", t.getName(), r, nanos);
    }

    protected void afterExecute(Runnable r, Throwable t) {
        super.afterExecute(r, t);
        // Record end time
        long nanos = System.nanoTime();
        long total = nanos - startNanos.get();

        LOGGER.info("Thread {} with task {} took {} nanos", t.getName(), r, totals);
    }
}

### Implementation
Below is a simple implementation of thread pool that accepts tasks:

In [ ]:
class SimpleThreadPoolExecutor {
    private final List<Worker> workers;
    private final Queue<Runnable> tasks;
    private volatile boolean isShutdown = false;

    public SimpleThreadPool(int nThreads) {
        workers = new ArrayList<>(nThreads);
        tasks = new ArrayDeque<>();
        for (int i = 0; i < nThreads; i++) {
            Worker worker = new Worker();
            workers.add(worker);
            worker.start();
        }
    }

    public void execute(Runnable task) {
        Objects.requireNonNull(task);

        synchronized (tasks) {
            if (isShutdown) {
                throw new IllegalStateException("ThreadPool is shutdown");
            }

            tasks.add(task);
            tasks.notify();
        }
    }

    public boolean awaitTermination(long timeout, TimeUnit unit) throws InterruptedException {
        long nanos = unit.toNanos(timeout);
        long deadline = System.nanoTime() + nanos;
    
        for (Worker worker : workers) {
            long remaining = deadline - System.nanoTime();
            if (remaining <= 0) {
                return false; // timeout before this worker finished
            }
    
            // Convert remaining time to millis for join
            long millis = TimeUnit.NANOSECONDS.toMillis(remaining);
            int nanosPart = (int)(remaining - TimeUnit.MILLISECONDS.toNanos(millis));
    
            // timed join
            worker.join(millis, nanosPart);
    
            // If worker.isAlive() after the timed join, timeout occurred
            if (worker.isAlive()) {
                return false;
            }
        }
    
        return true; // all workers terminated before timeout
    }

    public void shutdown() {
        synchronized (tasks) {
            isShutdown = true;
            tasks.notifyAll();
        }
    }

    private class Worker extends Thread {
        @Override
        public void run() {
            while(true) {
                Runnable task = null;
                synchronized (tasks) {
                    // Wait until a task is available
                    while (tasks.isEmpty() && !isShutdown) {
                        try {
                            tasks.wait();
                        } catch (InterruptedException e) {
                            e.printStackTrace();
                        }
                    }

                    if (tasks.isEmpty() && isShutdown) {
                        break;
                    }

                    task = tasks.poll();
                }

                try {
                    task.run();
                } catch (Exception e) {
                    e.printStackTrace();
                }
            }
        }
    }
}

## ForkJoinPool
A special `ExecutorService` suited for computationally intensive tasks that can be broken down into smaller tasks and run parallely. 

`ForkJoinPool` utilises *work-stealing* algorithm which means once a thread finishes a task, it steals task from other threads thereby ensuring that no thread sits idle. Every thread in the `ForkJoinPool` has its own doubly-ended queue. A thread picks tasks from its own queue to execute from one end, whereas other threads can pick tasks from the other end (if that thread's own work queue is empty).  

<img src="images/work_stealing.png" width=700 height=auto />

Task to be executed by the `ForkJoinPool` is represented using either:
- `RecursiveAction`: `compute` method doesn't return a value
- `RecursiveTask`: `compute` method returns value

Sample example showing usage of `RecursiveTask`:

In [ ]:
class SummationTask extends RecursiveTask<Integer> {
    private int[] array;
    private int start;
    private int end;
    private static final int THRESHOLD = 10;

    public SummationTask(int[] array, int start, int end) {
        this.array = array;
        this.start = start;
        this.end = end;
    }

    @Override
    protected Integer compute() {
        if (end - start < THRESHOLD) {
            int sum = 0;
            for (int i = start; i < end; i++) {
                sum += array[i];
            }
            return sum;
        } else {
            int mid = (start + end) / 2;
            SummationTask leftTask = new SummationTask(array, start, mid);
            SummationTask rightTask = new SummationTask(array, mid, end);

            // Fork left, thereby adding to the thread's queue
            // Some other thread may steal it
            leftTask.fork();
            // Compute right in the same thread
            int rightSum = rightTask.compute();
            int leftSum = leftTask.join();
            return leftSum + rightSum;
        }
    }
}

ForkJoinPool forkJoinPool = new ForkJoinPool();
int[] array = IntStream.range(0, 1000000).toArray();
System.out.println(forkJoinPool.invoke(new SummationTask(array, 0, array.length - 1)));
forkJoinPool.shutdown();

Same computation, but with `RecursiveAction`:

In [ ]:
class SummationTask extends RecursiveAction {
    private int[] array;
    private int start;
    private int end;
    public AtomicInteger result;
    private static final int THRESHOLD = 10;

    public SummationTask(int[] array, int start, int end, AtomicInteger result) {
        this.array = array;
        this.start = start;
        this.end = end;
        this.result = result;
    }

    @Override
    protected void compute() {
        if (end - start < THRESHOLD) {
            int sum = 0;
            for (int i = start; i < end; i++) {
                sum += array[i];
            }
            result.addAndGet(sum);
        } else {
            int mid = (start + end) / 2;
            SummationTask leftTask = new SummationTask(array, start, mid, result);
            SummationTask rightTask = new SummationTask(array, mid, end, result);

            leftTask.fork();
            rightTask.compute();
            leftTask.join();
        }
    }
}

ForkJoinPool forkJoinPool = new ForkJoinPool(4);
int[] array = IntStream.range(0, 1000000).toArray();
AtomicInteger result = new AtomicInteger(0);
SummationTask task = new SummationTask(array, 0, array.length - 1, result);
forkJoinPool.invoke(task);
forkJoinPool.shutdown();
System.out.println(result);

`ForkJoinPool` maintains its own threads, the number can be passed to the constructor:

In [ ]:
ForkJoinPool fP1 = new ForkJoinPool(); // Runtime.availableProcessors by default
ForkJoinPool fP2 = new ForkJoinPool(4); // 4 = targeted number of active threads doing work in parallel
                                        // if all threads are blocked, then more can be created to maintain
                                        // stated parallelism

## CompletableFuture
Is a `Future` to which we can register actions to be performed once the value is available without blocking. Here is the `Future` class hierarchy for reference:  

<img src="images/future_hierarchy.png" width=700 height=auto />

We can create a `CompletableFuture` in multiple ways. For example, to create an already completed future:

In [3]:
import java.util.concurrent.*;

CompletableFuture<String> successFuture = CompletableFuture.completedFuture("In success state");
System.out.println("Future is done=" + successFuture.isDone() + ", it resolves to " + successFuture.get());

CompletableFuture<String> failedFuture = CompletableFuture.failedFuture(new RuntimeException("In failed state"));
System.out.println("Future is done=" + failedFuture.isDone() + ", it is in failed state");

Future is done=true, it resolves to In success state
Future is done=true, it is in failed state


The most common way is to use `runAsync`/`supplyAsync` methods:

In [5]:
// Runs the computation in ForkJoinPool.common. Accepts a Supplier
CompletableFuture<String> cF1 = CompletableFuture.supplyAsync(() -> "Running in thread " + Thread.currentThread().getName());
System.out.println(cF1.get());
// Runs the computation in custom executors
ThreadFactory factory = Thread.ofVirtual()
        .name("vt-", 0)   // prefix "vt-", number auto-incremented
        .factory();
ExecutorService executor = Executors.newThreadPerTaskExecutor(factory);
CompletableFuture<String> cF2 = CompletableFuture.supplyAsync(() -> "Running in thread " + Thread.currentThread().getName(), executor);
System.out.println(cF2.get());

Running in thread ForkJoinPool.commonPool-worker-1
Running in thread vt-0


In [6]:
// Accepts a Runnable
CompletableFuture<Void> cF3 = CompletableFuture.runAsync(() -> System.out.println("Running in thread " + Thread.currentThread().getName()));
System.out.println(cF3.get());

Running in thread ForkJoinPool.commonPool-worker-2
null


We can combine multiple `CompletableFuture`s using `allOf` or `anyOf`:

In [ ]:
CompletableFuture<String> first = CompletableFuture.supplyAsync(() -> {
    System.out.println("Running in thread " + Thread.currentThread().getName());
    return "First completed";
});
CompletableFuture<String> second = CompletableFuture.supplyAsync(() -> {
    System.out.println("Running in thread " + Thread.currentThread().getName());
    return "Second completed";
});
CompletableFuture<String> third = CompletableFuture.supplyAsync(() -> {
    System.out.println("Running in thread " + Thread.currentThread().getName());
    return "Third completed";
});
CompletableFuture<Void> allOf = CompletableFuture.allOf(first, second, third);
allOf.join();
System.out.println(first.join() + " " + second.join() + " " + third.join());

/*
join()                   |         get()
-------------------------|--------------------------------------
throws unchecked         | throws checked InterruptionException
CompletionException      |  and ExecutionException
                         |                           
does not respond to      |  responds to interruption
interruption             |           
*/

### Composition
`CompletableFutures` can be chained together, which means we can register actions to be done when the parent future completes. All the methods discussed above return a `CompletableFuture`. There are over 60 methods than let us transform the previous future into another future. Some are listed below:
- `thenApply(Function<? super T,? extends U> fn)`: takes the result of the previous future and returns another `CompletableFutures` that *resolves* to another value.
- `thenAccept(Consumer<? super T> action)`: consumes the result of previous future and returns another `CompletableFutures` that resolves to `null`.
- `thenRun(Runnable r)`: ignores the result of previous future, resolves to `null`.

In [7]:
CompletableFuture<Void> completableFuture = CompletableFuture.supplyAsync(() -> {
    System.out.println("Running task in thread: " + Thread.currentThread().getName());
    return "paris";
}).thenApply(x -> {
    System.out.println("Running task in thread: " + Thread.currentThread().getName());
    return x.toUpperCase();
}).thenAccept(x -> {
    System.out.println("Running task in thread: " + Thread.currentThread().getName());
    System.out.println("Value obtained: " + x);
}).thenRun(() -> {
    System.out.println("Running task in thread: " + Thread.currentThread().getName());
    System.out.println("Task completed");
});

System.out.println("Starting execution");
completableFuture.join();

Running task in thread: ForkJoinPool.commonPool-worker-3
Running task in thread: ForkJoinPool.commonPool-worker-3
Running task in thread: ForkJoinPool.commonPool-worker-3
Value obtained: PARIS
Running task in thread: ForkJoinPool.commonPool-worker-3
Task completed
Starting execution


The above code could have also printed:
```
Running task in thread: ForkJoinPool.commonPool-worker-1
Running task in thread: ForkJoinPool.commonPool-worker-1
Running task in thread: main
Value obtained: PARIS
Running task in thread: main
Task completed
Starting execution
```

But why main? If a dependent stage is attached after its parent is already completed, it may run synchronously in the current calling thread. In this case `supplyAsync` and `thenApply` executed and moved to completed state before `thenAccept` was even registered. Therefore `thenAccept` ran on the calling thread which is main in this case.

Formally, `thenApply`/`thenAccept`/`thenRun` run in the same thread that completes the previous stage:
- If the previous stage completes in a `ForkJoin` worker, the continuation runs on that worker.
- If the previous stage completes in the main thread, the continuation runs on the main thread.
- If the stage is already completed by the time you attach the next stage, the continuation runs synchronously on the calling thread.

Java provides the following methods:
- `thenApplyAsync`
- `thenAcceptAsync`
- `thenRunAsync`
  
In these methods, the continuation runs:
- Either on the `ForkJoinPool.commonPool` (default), or
- On the executor you provide, if you pass one.

In [10]:
CompletableFuture<Void> completableFuture = CompletableFuture.supplyAsync(() -> {
    System.out.println("Running task in thread: " + Thread.currentThread().getName());
    return "paris";
}).thenApplyAsync(x -> {
    System.out.println("Running task in thread: " + Thread.currentThread().getName());
    return x.toUpperCase();
}).thenAcceptAsync(x -> {
    System.out.println("Running task in thread: " + Thread.currentThread().getName());
    System.out.println("Value obtained: " + x);
}).thenRunAsync(() -> {
    System.out.println("Running task in thread: " + Thread.currentThread().getName());
    System.out.println("Task completed");
});

System.out.println("Starting execution");
completableFuture.join();

Running task in thread: ForkJoinPool.commonPool-worker-5
Running task in thread: ForkJoinPool.commonPool-worker-5
Running task in thread: ForkJoinPool.commonPool-worker-5
Value obtained: PARIS
Running task in thread: ForkJoinPool.commonPool-worker-5
Task completed
Starting execution


What happens when any previous stage returns an exception. There are multiple ways to handle that, but it basically classified into two groups - either you want to recover from the exception (by handling the exception and returning another value) or you don't. The following methods are available:
- `exceptionally(Function<Throwable, ? extends T> fn)` and `exceptionallyAsync`: if we want to recover from an exception and return a replacement value.
- `handle(BiFunction<? super T, Throwable, ? extends U> fn)` and `handleAsync`: similar to the previous one, with the difference being you can access previous value if there was no exception.
- `whenComplete(BiConsumer<? super T, ? super Throwable> action)` and `whenCompleteAsync`: this version is used when we don't want to recover. Acts like finally block.

In [11]:
// The exception flows down
CompletableFuture<String> completableFuture = CompletableFuture
    .failedFuture(new RuntimeException("Error!"))
    .thenApplyAsync(x -> {
        System.out.println("In thenApplyAsync:");
        return "Hello";
    })
    .exceptionally(t -> {
        System.out.println(t.getMessage());
        return "World";
    });

completableFuture.join();

java.lang.RuntimeException: Error!


World

In [13]:
CompletableFuture<String> completableFuture = CompletableFuture
    .failedFuture(new RuntimeException("Error!"))
    .thenApplyAsync(x -> {
        System.out.println("In thenApplyAsync:");
        return "Hello";
    })
    .handleAsync((v, t) -> {
        if (t != null)
            System.out.println(t.getMessage());
        return v != null ? v.toUpperCase() : null;
    });

completableFuture.join();

java.lang.RuntimeException: Error!
